# Poetry Generation with RNNs

In [29]:
import numpy
import re
import os


import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [30]:
# Function from solutions in Set 6 to change text to data
def parse_observations(text):
    # Convert text to dataset.
    lines = [line.split() for line in text.split('\n') if line.split()]

    obs_counter = 0
    obs = []
    obs_map = {}

    for line in lines:
        obs_elem = []
        
        for word in line:
            word = re.sub(r'[^\w]', '', word).lower()
            if word not in obs_map:
                # Add unique words to the observations map.
                obs_map[word] = obs_counter
                obs_counter += 1
            
            # Add the encoded word.
            obs_elem.append(obs_map[word])
        
        # Add the encoded sequence.
        obs.append(obs_elem)

    return obs, obs_map


In [31]:
text = open(os.path.join(os.getcwd(), 'data/shakespeare.txt')).read()
obs, obs_map = parse_observations(text)

#chars = list(text)
chars = sorted(list(set(text)))
n_chars, n_vocab = len(text), len(chars)

char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 40
train = []
char_seqs = []

for i in range(0, n_chars - seq_length, 2):
    seq_in = text[i:i + seq_length]
    train.append(seq_in)
    
    if i + seq_length < n_chars - 1:
        seq_out = text[i + seq_length]
        char_seqs.append(seq_out)

n_patterns = len(train)
print("Total Patterns: ", n_patterns)

char_seqs.append(' ')

print(len(char_seqs))

Total Characters:  98029
Total Vocab:  71
Total Patterns:  48995
48995


In [32]:
# One-hot encode the training data

xTrain = np.zeros((n_patterns, seq_length, n_vocab))
yTrain = np.zeros((n_patterns, n_vocab))

for ind in range(n_patterns):
    sentence = train[ind]
    for i, c in enumerate(sentence):
        xTrain[ind, i, char_to_int[c]] = 1
    
    yTrain[ind, char_to_int[char_seqs[ind]]] = 1

In [33]:
model = Sequential()
model.add(LSTM(128, input_shape=(seq_length, n_vocab)))
model.add(Dropout(0.2))
model.add(Dense(n_vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.fit(xTrain, yTrain, epochs=60, batch_size=128)


Epoch 1/60
48995/48995 [==============================] - 72s - loss: 2.9674    
Epoch 2/60
48995/48995 [==============================] - 115s - loss: 2.4258   
Epoch 3/60
48995/48995 [==============================] - 84s - loss: 2.2312    
Epoch 4/60
48995/48995 [==============================] - 82s - loss: 2.1211    
Epoch 5/60
48995/48995 [==============================] - 70s - loss: 2.0507    
Epoch 6/60
48995/48995 [==============================] - 79s - loss: 1.9978    
Epoch 7/60
48995/48995 [==============================] - 59s - loss: 1.9498     
Epoch 8/60
48995/48995 [==============================] - 110s - loss: 1.9111   
Epoch 9/60
48995/48995 [==============================] - 96s - loss: 1.8746    
Epoch 10/60
48995/48995 [==============================] - 72s - loss: 1.8433    
Epoch 11/60
48995/48995 [==============================] - 79s - loss: 1.8144    
Epoch 12/60
48995/48995 [==============================] - 106s - loss: 1.7901   
Epoch 13/60
48995/48995 

In [37]:
start = numpy.random.randint(0, n_chars - seq_length - 1)
sentence = "shall i compare thee to a summer's day?\n"
seq_in = sentence
# generate characters
for ind in range(1000):
    x_format = np.zeros((1, seq_length, n_vocab))
    
    for i, c in enumerate(sentence):
        x_format[0, i, char_to_int[c]] = 1.0
    
    prediction = model.predict(x_format, verbose=0)
    
    pred_array = np.array(prediction)
    
    best_ind = np.argmax(pred_array[0])
    
    result = int_to_char[best_ind]
    
    sentence = sentence[1:] + result
    
    seq_in += result
#print( "\nDone.")
print(seq_in)

shall i compare thee to a summer's day?
The corell dot my self me sences in thee,
  This thing hat I am that I do couse,
  Then this should be ithers and true should hath and resile,
That hath my love this linge that thou art,
And to the eaver beauty of thy san,
That thes for when I am the love's foon dure.
The brat the surst of hear that fear betred,
  in thee to me fan thy self and thing allow,
The  we thou dead farth me for my self when self thought
To must this should that men mored sweet sor,
  And so seef I dear that I wort of thee,
The wordot soul stoul sump to thy ure's sing,
That have a condounded in the lorst
Thou hast a corfon my doth when thou art,
Which in thee my self and world with stan,
Whon thou art my heart world be thours in the distinge,
And that to the ead the world so best inge,
If any dost confornt,
That there a me my love soul store of thee,
  This thinge should this will thou art thee, my dodn and,
Time to the world sheer when I am you dot,
Then thou art the se